# 1.4 Bivariatna analiza – **klasifikacija** (Heart_Disease)


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from pathlib import Path

# -------------------------
# Nastavitve
# -------------------------
CSV_PATH = "CVD_cleaned.csv"  # privzeto ime datoteke
# fallback (če uporabljaš drugačno ime v repo):
if not Path(CSV_PATH).exists() and Path("CVD_cleaned_v2.csv").exists():
    CSV_PATH = "CVD_cleaned_v2.csv"   # če je datoteka v repo root; po potrebi popravi pot
TARGET = "Heart_Disease"
RANDOM_STATE = 42

SAVE_FIGS = True
FIG_DIR = Path("figures/1_4_classification")
FIG_DIR.mkdir(parents=True, exist_ok=True)

ALPHA = 0.05  # prag značilnosti


In [14]:
# -------------------------
# Nalaganje podatkov
# -------------------------
assert Path(CSV_PATH).exists(), f"Ne najdem datoteke: {CSV_PATH} (preveri pot/ime)"
df = pd.read_csv(CSV_PATH)

print("Shape:", df.shape)
display(df.head())

# Ciljna spremenljivka
print("\nPorazdelitev ciljne spremenljivke (Heart_Disease):")
display(df[TARGET].value_counts())
display((df[TARGET].value_counts(normalize=True)*100).round(3))


Shape: (308854, 19)


,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0



Porazdelitev ciljne spremenljivke (Heart_Disease):


Heart_Disease
No     283883
Yes     24971
Name: count, dtype: int64

Heart_Disease
No     91.915
Yes     8.085
Name: proportion, dtype: float64

## Logika izbire statističnih testov (po tipu spremenljivk)

### A) Kategorialna (X) vs. kategorialna (Y)
- Uporabimo **χ² test neodvisnosti**.
- Če je kontingenčna tabela **2×2** in so pričakovane frekvence **< 5**, uporabimo **Fisherjev eksaktni test**.
- Mera učinka: **Cramérjev V**.

### B) Numerična (X) vs. binarna kategorialna (Y)
Ker ima `Heart_Disease` dve skupini (`Yes`/`No`), primerjamo porazdelitev numerične spremenljivke med skupinama:

1. Preverimo **normalnost** (Shapiro-Wilk na vzorcu do 5000 primerov na skupino).
2. Če je normalnost OK, preverimo **enakost varianc** (Levenov test na vzorcu).
3. Izbira testa:
   - normalno + enake variance → **Studentov t-test**
   - normalno + neenake variance → **Welchov t-test**
   - sicer → **Mann–Whitney U test** (neparametrični)

Mere učinka:
- t-test → **Cohenov d**
- Mann–Whitney → **rank-biserial r**

### Grafi
- numerične spremenljivke: **boxplot** po skupinah `Heart_Disease`
- kategorialne spremenljivke: **delež “Yes” po kategorijah** (barplot)


In [18]:
# Urejanje vrstnega reda za ordinalne kategorije (da so barploti "po vrsti")
CATEGORY_ORDER = {
    "General_Health": ["Poor", "Fair", "Good", "Very Good", "Excellent"],
    "Checkup": [
        "Within the past year",
        "Within the past 2 years",
        "Within the past 5 years",
        "5 or more years ago",
        "Never",
    ],
    "Age_Category": [
        "18-24", "25-29", "30-34", "35-39",
        "40-44", "45-49", "50-54", "55-59",
        "60-64", "65-69", "70-74", "75-79", "80+"
    ],
}


In [19]:
# -------------------------
# Pomožne funkcije
# -------------------------
def safe_name(s: str) -> str:
    # Za varna imena datotek (odstrani posebne znake)
    return (
        s.replace("(", "")
         .replace(")", "")
         .replace("/", "_")
         .replace("\\", "_")
         .replace(" ", "_")
         .replace("__", "_")
         .replace(":", "")
         .replace("%", "pct")
    )

def _sample_series(s: pd.Series, n: int, random_state: int = 42) -> pd.Series:
    s = s.dropna()
    if len(s) <= n:
        return s
    return s.sample(n=n, random_state=random_state)

def normality_p(s: pd.Series, max_n: int = 5000, random_state: int = 42) -> float:
    # Shapiro-Wilk normalnost na vzorcu (do max_n)
    s2 = _sample_series(s, max_n, random_state)
    if len(s2) < 3:
        return np.nan
    return float(stats.shapiro(s2)[1])

def levene_p(g1: pd.Series, g2: pd.Series, max_n: int = 5000, random_state: int = 42) -> float:
    # Levenov test enakosti varianc (na vzorcih)
    a = _sample_series(g1, max_n, random_state).values
    b = _sample_series(g2, max_n, random_state).values
    if len(a) < 2 or len(b) < 2:
        return np.nan
    return float(stats.levene(a, b, center="median")[1])

def cramers_v(ct: pd.DataFrame) -> float:
    # Cramérjev V (mera učinka) za χ² test
    chi2 = stats.chi2_contingency(ct, correction=False)[0]
    n = ct.values.sum()
    r, k = ct.shape
    if n == 0 or min(r, k) <= 1:
        return np.nan
    return float(np.sqrt((chi2 / n) / (min(r - 1, k - 1))))

def cohens_d(x: pd.Series, y: pd.Series) -> float:
    # Cohenov d (mera učinka) za dve skupini
    x = x.dropna().astype(float)
    y = y.dropna().astype(float)
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return np.nan
    sx, sy = x.std(ddof=1), y.std(ddof=1)
    sp = np.sqrt(((nx - 1) * sx**2 + (ny - 1) * sy**2) / (nx + ny - 2))
    if sp == 0:
        return np.nan
    return float((x.mean() - y.mean()) / sp)

def rank_biserial_from_u(u: float, n1: int, n2: int) -> float:
    # Rank-biserial r iz Mann-Whitney U
    if n1 == 0 or n2 == 0:
        return np.nan
    return float(1 - (2*u)/(n1*n2))

def plot_box_by_binary(x: pd.Series, y: pd.Series, x_name: str, y_name: str, save_path=None):
    # Boxplot numerične spremenljivke po binarni ciljki
    y_clean = y.dropna()
    levels = ["No", "Yes"] if set(y_clean.unique()) == {"No","Yes"} else sorted(y_clean.unique(), key=str)
    data = [x[y == lvl].dropna().astype(float).values for lvl in levels]

    plt.figure()
    plt.boxplot(data, labels=[str(l) for l in levels], vert=True, showfliers=True)
    plt.title(f"{x_name} po {y_name}")
    plt.xlabel(y_name)
    plt.ylabel(x_name)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=200)
        plt.close()
    else:
        plt.show()

def plot_yes_rate_by_category(x: pd.Series, y: pd.Series, x_name: str, y_name: str, save_path=None, max_cats: int = 30, order=None):
    # Barplot: delež pozitivnega razreda (Yes) po kategorijah
    tmp = pd.DataFrame({x_name: x, y_name: y}).dropna()
    y_bin = tmp[y_name].map({"Yes": 1, "No": 0})
    tmp = tmp.assign(_ybin=y_bin)

    grp = tmp.groupby(x_name)["_ybin"].agg(["mean","count"]).sort_values("count", ascending=False)

    # Če imamo podan vrstni red kategorij (ordinalne spremenljivke), ga uporabimo za smiselnejši graf
    if order is not None:
        order_present = [o for o in order if o in grp.index]
        rest = [i for i in grp.index if i not in order_present]
        grp = grp.loc[order_present + rest]

    if len(grp) > max_cats:
        grp = grp.head(max_cats)

    plt.figure(figsize=(8, 4))
    plt.bar(grp.index.astype(str), grp["mean"]*100)
    plt.title(f"Delež '{y_name}=Yes' po {x_name}")
    plt.ylabel("Delež Yes (%)")
    plt.xticks(rotation=45, ha="right")
    for i, (idx, row) in enumerate(grp.iterrows()):
        plt.text(i, row["mean"]*100, f"n={int(row['count'])}", ha="center", va="bottom", fontsize=8, rotation=90)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=200)
        plt.close()
    else:
        plt.show()


In [20]:
# -------------------------
# Tipi spremenljivk (v tem naboru)
# -------------------------
numeric_cols = [
    "Height_(cm)",
    "Weight_(kg)",
    "BMI",
    "Alcohol_Consumption",
    "Fruit_Consumption",
    "Green_Vegetables_Consumption",
    "FriedPotato_Consumption",
]

categorical_cols = [c for c in df.columns if c not in numeric_cols + [TARGET]]

print("Numerične:", numeric_cols)
print("Kategorialne:", categorical_cols)


Numerične: ['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption', 'Fruit_Consumption', 'Green_Vegetables_Consumption', 'FriedPotato_Consumption']
Kategorialne: ['General_Health', 'Checkup', 'Exercise', 'Skin_Cancer', 'Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex', 'Age_Category', 'Smoking_History']


In [21]:
# -------------------------
# Bivariatna analiza: X vs Heart_Disease
# -------------------------
results = []
y = df[TARGET]

# 1) NUMERIČNE: t-test / Welch / Mann-Whitney
for x_name in numeric_cols:
    x = df[x_name]
    g_no  = x[y == "No"]
    g_yes = x[y == "Yes"]

    # normalnost na vzorcih
    p_norm_no  = normality_p(g_no,  max_n=5000, random_state=RANDOM_STATE)
    p_norm_yes = normality_p(g_yes, max_n=5000, random_state=RANDOM_STATE)
    normal = (p_norm_no > ALPHA) and (p_norm_yes > ALPHA)

    # enakost varianc
    p_lev = levene_p(g_no, g_yes, max_n=5000, random_state=RANDOM_STATE)
    equal_var = (p_lev > ALPHA) if not np.isnan(p_lev) else False

    if normal and equal_var:
        stat, pval = stats.ttest_ind(g_no, g_yes, equal_var=True, nan_policy="omit")
        test_name = "Studentov t-test"
        effect = cohens_d(g_no, g_yes)
        effect_name = "Cohenov d"
        note = f"Normalnost OK (p={p_norm_no:.3f}, {p_norm_yes:.3f}); Levene p={p_lev:.3f}."
    elif normal and not equal_var:
        stat, pval = stats.ttest_ind(g_no, g_yes, equal_var=False, nan_policy="omit")
        test_name = "Welchov t-test"
        effect = cohens_d(g_no, g_yes)
        effect_name = "Cohenov d"
        note = f"Normalnost OK; varianca neenaka (Levene p={p_lev:.3f})."
    else:
        stat, pval = stats.mannwhitneyu(g_no.dropna(), g_yes.dropna(), alternative="two-sided", method="auto")
        test_name = "Mann–Whitney U"
        effect = rank_biserial_from_u(stat, g_no.dropna().shape[0], g_yes.dropna().shape[0])
        effect_name = "Rank-biserial r"
        note = f"Normalnost ni OK (p={p_norm_no:.3f}, {p_norm_yes:.3f}) → neparametrični test."

    if SAVE_FIGS:
        pth = FIG_DIR / f"box_{safe_name(x_name)}.png"
        plot_box_by_binary(x, y, x_name, TARGET, pth)

    results.append({
        "X": x_name,
        "Tip X": "numerična",
        "Test": test_name,
        "Statistika": float(stat),
        "p": float(pval),
        "Mera učinka": effect_name,
        "Učinek": float(effect),
        "Opomba": note,
        "N_No": int(g_no.notna().sum()),
        "N_Yes": int(g_yes.notna().sum()),
        "No_mean": float(g_no.mean()),
        "Yes_mean": float(g_yes.mean()),
        "No_median": float(g_no.median()),
        "Yes_median": float(g_yes.median()),
    })

# 2) KATEGORIALNE: χ² / Fisher
for x_name in categorical_cols:
    x = df[x_name]
    ct = pd.crosstab(x, y)

    chi2, p_chi, dof, expected = stats.chi2_contingency(ct, correction=False)
    exp_min = float(np.min(expected))

    if ct.shape == (2, 2) and exp_min < 5:
        oddsratio, pval = stats.fisher_exact(ct.values)
        test_name = "Fisherjev eksaktni test (2×2)"
        stat = oddsratio
        note = f"Pričakovane frekvence < 5 (min={exp_min:.3f})."
        effect = np.nan
        effect_name = "—"
    else:
        test_name = "χ² test neodvisnosti"
        stat = chi2
        pval = p_chi
        note = f"dof={dof}, min pričakovane={exp_min:.3f}"
        effect = cramers_v(ct)
        effect_name = "Cramérjev V"

    if SAVE_FIGS:
        pth = FIG_DIR / f"bar_{safe_name(x_name)}.png"
        plot_yes_rate_by_category(x, y, x_name, TARGET, pth, order=CATEGORY_ORDER.get(x_name))

    results.append({
        "X": x_name,
        "Tip X": "kategorialna",
        "Test": test_name,
        "Statistika": float(stat),
        "p": float(pval),
        "Mera učinka": effect_name,
        "Učinek": float(effect) if effect == effect else np.nan,
        "Opomba": note,
        "N_No": int((y == "No").sum()),
        "N_Yes": int((y == "Yes").sum()),
        "No_mean": np.nan,
        "Yes_mean": np.nan,
        "No_median": np.nan,
        "Yes_median": np.nan,
    })

biv_table = pd.DataFrame(results)

biv_table["abs_effect"] = biv_table["Učinek"].abs()
biv_table = biv_table.sort_values(["p", "abs_effect"], ascending=[True, False]).reset_index(drop=True)

display(biv_table.head(10))


C:\Users\krist\AppData\Local\Temp\ipykernel_6832\1576448644.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=[str(l) for l in levels], vert=True, showfliers=True)
C:\Users\krist\AppData\Local\Temp\ipykernel_6832\1576448644.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=[str(l) for l in levels], vert=True, showfliers=True)
C:\Users\krist\AppData\Local\Temp\ipykernel_6832\1576448644.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=[str(l) for l in levels], vert=True, showfliers=True)
C:\Users\krist\AppData\Local\Temp\ipykernel_6832\

,X,Tip X,Test,Statistika,p,Mera učinka,Učinek,Opomba,N_No,N_Yes,No_mean,Yes_mean,No_median,Yes_median,abs_effect
0,General_Health,kategorialna,χ² test neodvisnosti,1.923433e+04,0.0,Cramérjev V,0.249553,"dof=4, min pričakovane=916.117",283883,24971,NaN,NaN,NaN,NaN,0.249553
1,Age_Category,kategorialna,χ² test neodvisnosti,1.803593e+04,0.0,Cramérjev V,0.241653,"dof=12, min pričakovane=1252.698",283883,24971,NaN,NaN,NaN,NaN,0.241653
2,Diabetes,kategorialna,χ² test neodvisnosti,1.041855e+04,0.0,Cramérjev V,0.183665,"dof=3, min pričakovane=213.930",283883,24971,NaN,NaN,NaN,NaN,0.183665
3,Arthritis,kategorialna,χ² test neodvisnosti,7.316549e+03,0.0,Cramérjev V,0.153913,"dof=1, min pričakovane=8171.641",283883,24971,NaN,NaN,NaN,NaN,0.153913
4,Alcohol_Consumption,numerična,Mann–Whitney U,4.082698e+09,0.0,Rank-biserial r,-0.151866,"Normalnost ni OK (p=0.000, 0.000) → neparametr...",283883,24971,5.185298,4.085339,1.0,0.0,0.151866
5,Smoking_History,kategorialna,χ² test neodvisnosti,3.588941e+03,0.0,Cramérjev V,0.107797,"dof=1, min pričakovane=10127.657",283883,24971,NaN,NaN,NaN,NaN,0.107797
6,Exercise,kategorialna,χ² test neodvisnosti,2.867001e+03,0.0,Cramérjev V,0.096347,"dof=1, min pričakovane=5616.927",283883,24971,NaN,NaN,NaN,NaN,0.096347
7,Checkup,kategorialna,χ² test neodvisnosti,2.731044e+03,0.0,Cramérjev V,0.094035,"dof=4, min pričakovane=113.757",283883,24971,NaN,NaN,NaN,NaN,0.094035
8,Other_Cancer,kategorialna,χ² test neodvisnosti,2.636184e+03,0.0,Cramérjev V,0.092387,"dof=1, min pričakovane=2415.651",283883,24971,NaN,NaN,NaN,NaN,0.092387
9,Skin_Cancer,kategorialna,χ² test neodvisnosti,2.549111e+03,0.0,Cramérjev V,0.090848,"dof=1, min pričakovane=2425.030",283883,24971,NaN,NaN,NaN,NaN,0.090848


In [22]:
# -------------------------
# Hitri povzetek ključnih povezav (za interpretacijo v poročilu)
# -------------------------
# Top kategorialne po Cramérjevem V
cat = biv_table[(biv_table["Tip X"]=="kategorialna") & (biv_table["Mera učinka"]=="Cramérjev V")].copy()
top_cat = cat.sort_values("Učinek", ascending=False).head(5)

print("TOP 5 kategorialnih spremenljivk (po Cramérjev V):")
display(top_cat[["X","Učinek","p","Test","Opomba"]])

# Za vsako top kategorialno pokažemo kategorije z najvišjim deležem Heart_Disease=Yes
for x_name in top_cat["X"]:
    tmp = df[[x_name, TARGET]].dropna()
    rate = tmp.assign(_y=tmp[TARGET].map({"Yes":1,"No":0})).groupby(x_name)["_y"].agg(["mean","count"])
    rate = rate.sort_values("mean", ascending=False).head(5)
    print(f"\n{x_name}: kategorije z najvišjim deležem Heart_Disease=Yes")
    display((rate.assign(Yes_pct=(rate["mean"]*100).round(2))
                 .drop(columns=["mean"])
                 .rename(columns={"count":"n"})))

# Top numerične po |rank-biserial r| ali |Cohen d|
num = biv_table[biv_table["Tip X"]=="numerična"].copy()
num["abs_eff"] = num["Učinek"].abs()
top_num = num.sort_values("abs_eff", ascending=False).head(5)

print("\nTOP 5 numeričnih spremenljivk (po absolutni meri učinka):")
display(top_num[["X","Učinek","Mera učinka","p","Test","No_median","Yes_median","No_mean","Yes_mean"]])


TOP 5 kategorialnih spremenljivk (po Cramérjev V):


,X,Učinek,p,Test,Opomba
0,General_Health,0.249553,0.0,χ² test neodvisnosti,"dof=4, min pričakovane=916.117"
1,Age_Category,0.241653,0.0,χ² test neodvisnosti,"dof=12, min pričakovane=1252.698"
2,Diabetes,0.183665,0.0,χ² test neodvisnosti,"dof=3, min pričakovane=213.930"
3,Arthritis,0.153913,0.0,χ² test neodvisnosti,"dof=1, min pričakovane=8171.641"
5,Smoking_History,0.107797,0.0,χ² test neodvisnosti,"dof=1, min pričakovane=10127.657"



General_Health: kategorije z najvišjim deležem Heart_Disease=Yes


,n,Yes_pct
General_Health,,
Poor,11331,31.79
Fair,35810,18.96
Good,95364,9.06
Very Good,110395,4.37
Excellent,55954,1.99



Age_Category: kategorije z najvišjim deležem Heart_Disease=Yes


...

,n,Yes_pct
Arthritis,,
Yes,101071,14.10
No,207783,5.16



Smoking_History: kategorije z najvišjim deležem Heart_Disease=Yes


,n,Yes_pct
Smoking_History,,
Yes,125264,11.64
No,183590,5.66



TOP 5 numeričnih spremenljivk (po absolutni meri učinka):


,X,Učinek,Mera učinka,p,Test,No_median,Yes_median,No_mean,Yes_mean
4,Alcohol_Consumption,-0.151866,Rank-biserial r,0.000000e+00,Mann–Whitney U,1.00,0.00,5.185298,4.085339
11,Weight_(kg),0.109072,Rank-biserial r,2.695694e-180,Mann–Whitney U,81.19,83.91,83.298262,86.889986
12,BMI,0.106681,Rank-biserial r,1.934898e-172,Mann–Whitney U,27.44,28.55,28.543676,29.564505
14,Green_Vegetables_Consumption,-0.057551,Rank-biserial r,3.777408e-52,Mann–Whitney U,12.00,10.00,15.216878,13.900404
15,FriedPotato_Consumption,-0.049621,Rank-biserial r,1.863347e-39,Mann–Whitney U,4.00,4.00,6.320104,6.029594


In [23]:
# -------------------------
# Izvoz skupne tabele
# -------------------------
OUT_CSV = "1_4_bivariatna_analiza_klasifikacija.csv"
biv_table.drop(columns=["abs_effect"]).to_csv(OUT_CSV, index=False)
print("Shranjeno:", OUT_CSV)

print("\nSlike so shranjene v mapi:", FIG_DIR)


Shranjeno: 1_4_bivariatna_analiza_klasifikacija.csv

Slike so shranjene v mapi: figures\1_4_classification


## Kako interpretirati rezultate

- **p-vrednost**: statistična značilnost povezave (pri velikem N bo pogosto zelo majhna).
- **mera učinka** (bolj “procesno” relevantno):
  - **Cramérjev V**: ~0 (zanemarljivo), ~0.1 (majhno), ~0.3 (srednje), ~0.5 (veliko) *(pravilo palca)*.
  - **Cohenov d**: ~0.2 (majhen), ~0.5 (srednji), ~0.8 (velik).
  - **rank-biserial r**: blizu 0 (majhen), proti ±1 (velik).

V nadaljevanju (1.5 Feature Selection) lahko kot kandidatske spremenljivke vzamemo tiste z:
- najnižjimi p-vrednostmi **in/ali**
- največjimi absolutnimi merami učinka.
